
# UP Contingent Planning Tutorial

Contingent planning under partial observability and sensing actions is an importat problem in automated planning.
This notebook provides examples on using the contingent planning package of the unified planning framework. The package supports offline planning, where a complete plan graph is constructed, and online planning, where the planner interacts with the environment during execution, receving observations and computing which action to perfrom next.

For information about contingent planning, and the algorithms used here can be found at:


*   Shlomi Maliah, Radimir Komarnitsky, Guy Shani: Computing Contingent Plan Graphs using Online Planning. JAAMAS 16(1): 1:1-1:30 (2021)
*   Ronen I. Brafman, Guy Shani: Replanning in Domains with Partial Information and Sensing Actions. J. Artif. Intell. Res. 45: 565-600 (2012)

For questions or comments please contact Guy Shani - shanigu@bgu.ac.il.


# If you would like that the solution will be print change the parameter to Ture

In [1]:
SOLUTION_PRINTED = False

def print_CPOR_sol(p_planNode):
    if p_planNode is not None:
        x = p_planNode
        print(x.action_instance)
        for c in x.children:
            print_CPOR_sol(c[1])

### Installing the UP repo

We begin by installing the UP framework.

In [2]:
!pip install git+https://github.com/aiplan4eu/unified-planning@action-selector-mixin

  Cloning https://github.com/aiplan4eu/unified-planning (to revision action-selector-mixin) to c:\users\hilak\appdata\local\temp\pip-req-build-fxb97_ci
  Resolved https://github.com/aiplan4eu/unified-planning to commit 0a6fe4420117e6b46f9e50162e846e5a34bb301c
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/aiplan4eu/unified-planning 'C:\Users\hilak\AppData\Local\Temp\pip-req-build-fxb97_ci'
  Running command git checkout -b action-selector-mixin --track origin/action-selector-mixin
  Branch 'action-selector-mixin' set up to track remote branch 'action-selector-mixin' from 'origin'.
  Switched to a new branch 'action-selector-mixin'


### Installing the CPOR repo

We now install the CPOR repo of UP.

In [3]:
!pip install --upgrade --force-reinstall git+https://github.com/aiplan4eu/up-cpor

  Cloning https://github.com/aiplan4eu/up-cpor to c:\users\hilak\appdata\local\temp\pip-req-build-k03gan4s
  Resolved https://github.com/aiplan4eu/up-cpor to commit 45197d7241983e811b5a67d91d167cb99fd04b63
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pythonnet-3.0.0-py3-none-any.whl (278 kB)
  Using cached clr_loader-0.2.5-py3-none-any.whl (51 kB)
  Using cached cffi-1.15.1-cp39-cp39-win_amd64.whl (179 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
  Created wheel for up-cpor: filename=up_cpor-1.0.0-py3-none-any.whl size=196261 sha256=8b9e4269ed9c45c19a21fdd31332cdda16fc08762d6bf2adcab951326458a1f2
  Stored in directory: C:\Users\hilak\AppData\Local\Temp\pip-ephem-wheel-cache-7_qwjhap\wheels\97\2c\b4\82d2589899eb49a45adc1a9d4149cfa9865d01ff87f9c71a50
Successfully built up-cpor
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.21
    Uninstalling pycparser-2.21:
      Su

  Running command git clone --filter=blob:none --quiet https://github.com/aiplan4eu/up-cpor 'C:\Users\hilak\AppData\Local\Temp\pip-req-build-k03gan4s'
Reason for being yanked: Is missing python_requires and thus installs on unsupported versions


# Loading Problems

We are now done with installations, and can start defining a problem that the planner can tackle. In this tutorial we demonstrate how problems can be loaded from pddl, but one can define a contingent problem through other methods, using the UP API.

In [4]:
from unified_planning.io import PDDLReader

# Creating a PDDL reader
reader = PDDLReader()

prob_arr = ['blocks2', 'doors5', 'wumpus05']

# Offline Planning Example

We now deonstrate how to compute a complete plan graph for a contingent problem, where nodes are labeled by actions, and edges are labeled by observations. The package currently implements only the CPOR offline planner. We initialize the planner, and then call the solve method to compute a solution.

After a solution plan tree is computed, we can save the resulting plan to a file.

In [5]:
import unified_planning.environment as environment
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.engines.results import PlanGenerationResultStatus

for prob in prob_arr:
    print(f"###########################Problem: {prob} start###########################")
    # Parsing a PDDL problem from file
    problem = reader.parse_problem(
        f"../Tests/{prob}/d.pddl",
        f"../Tests/{prob}/p.pddl"
    )

    env = environment.get_environment()
    env.factory.add_engine('CPORPlanning', 'up_cpor.engine', 'CPORImpl')

    with OneshotPlanner(name='CPORPlanning') as planner:
        result = planner.solve(problem)
        if SOLUTION_PRINTED:
            print_CPOR_sol(result.plan.root_node)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{planner.name} found a valid plan!')
            print(f'Success')
        else:
            print('No plan found!')

###########################Problem: blocks2 start###########################
NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up_cpor_new2\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: CPOR
  * Developers:  Guy Shani
  * Description: CPOR is an offline contingent planner.
  *               It computes a complete plan tree (or graph) where each node is labeled by an action, and edges are labeled by observations.
  *              The leaves of the plan tree correspond to goal states.

CPORPlanning found a valid plan!
Success
###########################Problem: doors5 start###########################
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up_cpor_new2\lib\site-packages\unif

# Using a UP Classical Planner Inside CPOR

CPOR (and SDR) operate by creating classical planning problems that model the partial knowledge, and solve them, to obtain a heuristic about which action to choose next. The CPOR package contains an internal impementation of the popular FF classical planner, by Joerg Hoffman. However, the package supports running any UP classical solver. We demonstrate here how the UP implementation of Tamer can be used instead of the internal FF.

In [6]:
!pip install unified-planning[tamer]

In [7]:
import unified_planning.environment as environment
from unified_planning.engines.results import PlanGenerationResultStatus
from unified_planning.shortcuts import OneshotPlanner

for prob in prob_arr:
    print(f"###########################Problem: {prob} start###########################")
    # Parsing a PDDL problem from file
    problem = reader.parse_problem(
        f"../Tests/{prob}/d.pddl",
        f"../Tests/{prob}/p.pddl"
    )

    env = environment.get_environment()
    env.factory.add_meta_engine('MetaCPORPlanning', 'up_cpor.engine', 'CPORMetaEngineImpl')

    with OneshotPlanner(name='MetaCPORPlanning[tamer]') as planner:
        result = planner.solve(problem)
        if SOLUTION_PRINTED:
            print_CPOR_sol(result.plan.root_node)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{planner.name} found a valid plan!')
            print(f'Success')
        else:
            print('No plan found!')

###########################Problem: blocks2 start###########################
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up_cpor_new2\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Conitngent Planning Algorithms
  * Developers:  Guy Shani
  * Description: Algorithms for offline and online decision making under partial observability and sensing actions

CPORPlanning[Tamer] found a valid plan!
Success
###########################Problem: doors5 start###########################
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 521 of `C:\Users\hilak\anaconda3\envs\up_cpor_new2\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Conitngent Planning Algorithms
  * Developers:  Guy Shani
  * Description: Algorithms for offline and online decision making under partial observability and sensing acti

#Online Contingent Planning

While in offline planning the planner computes a complete plan graph, in online planning we take a closed loop approach, where an agent interacts with the environment during execution.

The agent executes an action in the environment, and then receives an observation as a result in the action. In goal-based contingent planning this loop continues until the agent ensures that the goal has been achieved.

The CPOR package implements the SDR contingent (re)planner. SDR operates by translating the contingent problem into a classical problem, solving it using a classical solver, and then executing the resulting actions, if their preconditions hold. When an unexpected observation was received, SDR replans.

The code below demonstrates how SDR can be used, interacting with a simulated environment, which is also implemented inside the CPOR package. The while loop below implements the closed loop process.



In [8]:
import unified_planning.environment as environment
from unified_planning.shortcuts import ActionSelector
from up_cpor.simulator import SDRSimulator

for prob in prob_arr:
    print(f"###########################Problem: {prob} start###########################")
    # Parsing a PDDL problem from file
    problem = reader.parse_problem(
        f"../Tests/{prob}/d.pddl",
        f"../Tests/{prob}/p.pddl"
    )

    env = environment.get_environment()
    env.factory.add_engine('SDRPlanning', 'up_cpor.engine', 'SDRImpl')

    with ActionSelector(name='SDRPlanning', problem=problem) as solver:
        simulatedEnv = SDRSimulator(problem)
        while not simulatedEnv.is_goal_reached():
            action = solver.get_action()
            observation = simulatedEnv.apply(action)
            solver.update(observation)
            if SOLUTION_PRINTED:
                print(f"Action: {action}\n Observation: {observation}")
        print(f'{solver.name} found a valid plan!')
        print(f'Success')

###########################Problem: blocks2 start###########################
  *** Credits ***
  * In operation mode `ActionSelector` at line 711 of `C:\Users\hilak\anaconda3\envs\up_cpor_new2\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: SDR
  * Developers:  Guy Shani
  * Description: SDR is an online contingent replanner.
  *              It provides one action at a time, and then awaits to receive an observation from the environment.

SDRPlanning found a valid plan!
Success
###########################Problem: doors5 start###########################
  *** Credits ***
  * In operation mode `ActionSelector` at line 711 of `C:\Users\hilak\anaconda3\envs\up_cpor_new2\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: SDR
  * Developers:  Guy Shani
  * Description: SDR is an online contingent replanner.
  *              It provides one action at a time, and then a